# Desafio Gerando Falções
## Análise Exploratória

Este notebook contém as analises realizadas durante a exploração dos dados.

Essas análises, serviram de base para o desenvolvimento do ETL do projeto.

### Importações

In [0]:
import datetime
import re

import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql.functions as F

from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import IntegerType, StringType, FloatType, DoubleType, DateType, TimestampType, StructField, StructType

### Obtenção dos Dados

In [0]:

def create_delta_table_from_csv(table_name, csv_path, database="default"):
    try:
        # Check if the table already exists
        df = spark.read.format("delta").table(f"{database}.{table_name}")
        print(f"Table {database}.{table_name} already exists.")
    except AnalysisException:
        # Read the CSV file into a DataFrame
        df = spark.read.csv(csv_path, header=True, inferSchema=True)
        
        # Write the DataFrame to a Delta table
        df.write.format("delta").saveAsTable(f"{database}.{table_name}")
        print(f"Table {database}.{table_name} created from {csv_path}.")
    
    return df

# Define paths
clientes_csv_path = "/Users/bentofreitas/Documents/clientes.csv"
produtos_csv_path = "/Users/bentofreitas/Documents/produtos.csv"
vendas_csv_path = "/Users/bentofreitas/Documents/vendas.csv"

# Create Delta tables and get DataFrames
clientes_df = create_delta_table_from_csv("clientes", clientes_csv_path)
produtos_df = create_delta_table_from_csv("produtos", produtos_csv_path)
vendas_df = create_delta_table_from_csv("vendas", vendas_csv_path)

### Funções de Exploração

In [0]:
def show_table(df):
    print("First 5 rows:")
    display(df.limit(5))

    print("Schema:")
    df.printSchema()

    row_count = df.count()
    print(f"Number of rows: {row_count}")

In [0]:
def basic_eda(df):
    print("Contagem de linhas duplicadas: ")
    duplicate_rows_count = df.count() - df.dropDuplicates().count()
    print(duplicate_rows_count)

    print("\nValores Nulos por coluna: ")
    null_counts = df.select([
        F.col(c).isNull().cast("int").alias(c) for c in df.columns
    ]).groupBy().sum()
    display(null_counts)
    
    print("\nResumo estatístico por coluna:")
    display(df.describe())

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import NumericType

def graphical_eda(df, text_size=12):
    # Convert Spark DataFrame to Pandas DataFrame for plotting
    pdf = df.toPandas()
    
    # Set default text size for plots
    plt.rcParams.update({'font.size': text_size})
    
    # Identify numerical and categorical columns
    num_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, NumericType)]
    cat_cols = [field.name for field in df.schema.fields if not isinstance(field.dataType, NumericType)]
    
    # Plot 1: Distribution of numerical columns
    for col in num_cols:
        plt.figure(figsize=(15, 10))
        sns.histplot(pdf[col].dropna(), kde=True)
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()
    
    # Plot 2: Count plot for categorical columns
    for col in cat_cols:
        plt.figure(figsize=(15, 20))
        sns.countplot(y=pdf[col].dropna(), order=pdf[col].value_counts().index)
        plt.title(f'Count plot of {col}')
        plt.xlabel('Count')
        plt.ylabel(col)
        plt.show()
    
    # Plot 3: Correlation heatmap for numerical columns
    if len(num_cols) > 1:
        plt.figure(figsize=(10, 8))
        corr = pdf[num_cols].corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Correlation Heatmap')
        plt.show()

### Construção de Base Unificada

In [0]:
# Cria dataframe unificado a partir 
df_unificado = (
    vendas_df
    .join(produtos_df, "id_produto", "inner")
    .join(clientes_df, "id_cliente", "inner")
)

# Adiciona coluna ano-mes, com o ano e o mês da data de venda
df_unificado = df_unificado.withColumn("ano_mes_venda", F.date_format(F.col('data_venda'), "yyyy-MM"))

# Re-ordena e seleciona somente as colunas relevantes
# Exclusão das colunas marca e cidade, como demonstrado na análise exploratória
df_unificado = df_unificado.select(
    'id_venda', 'canal_venda', 'valor', 'data_venda', 'ano_mes_venda',
    'id_produto', 'nome_produto', 'descricao', 'marca',
    'id_cliente', 'nome', 'email', 'telefone', 'cidade', 'estado', 'data_cadastro'
)

# Display the joined dataframe
# display(df_unificado)

### Visualização Inicial

In [0]:
print("Tabela Clientes: ")
show_table(clientes_df)

print("\nTabela Produtos: ")
show_table(produtos_df)

print("\nTabela Vendas: ")
show_table(vendas_df)

In [0]:
print("Tabela Unificada: ")
show_table(df_unificado)
basic_eda(df_unificado)
# graphical_eda(df_unificado)

### Tratamento de Valores Nulos

In [0]:
def fill_missing_marca(df):
    # Extrair a primeira palavra da coluna nome_produto
    df = df.withColumn("first_word", F.split(F.col("nome_produto"), " ").getItem(0))
    
    # Preencher valores ausentes na coluna marca com a primeira palavra de nome_produto
    df = df.withColumn("marca", F.when(F.col("marca").isNull(), 
                                       F.col("first_word")
                                      ).otherwise(F.col("marca")))
    
    # Remover a coluna temporária first_word
    df = df.drop("first_word")

    # Renomear a coluna marca para categoria_produto
    df = df.withColumnRenamed('marca','categoria_produto')
    
    return df

In [0]:
df_unificado = fill_missing_marca(df_unificado)

### Tratamento Coluna Cidade

In [0]:
df_unificado.select('nome','cidade','estado').display()

##### Análise dos dados da coluna Cidade

A coluna 'cidade' apresenta registros que, na realidade, aparentam corresponder a sobrenomes dos clientes e não a localidades. 

Por não representar o atributo proposto e não agregar valor à análise, optou-se pela exclusão da coluna, visando manter o foco do desenvolvimento nos principais geradores de ganho dessa análise.

**Conlusão**: Remover coluna 'cidade' do dataset final.

In [0]:
df_unificado = df_unificado.drop('cidade')

### Enriquecimento dos dados

In [0]:
# Definição do schema para a UDF
schema_cat_brand_model = StructType([
    StructField("categoria", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("modelo", StringType(), True),
])

# Função UDF para extrair categoria, marca e modelo do nome do produto, com base em uma lista de BRANDS
@F.udf(returnType=schema_cat_brand_model)
def extract_cat_brand_model(nome_produto: str):
    if not nome_produto:
        return {"categoria": None, "marca": None, "modelo": None}

    toks = nome_produto.strip().split()
    marca_idx = None
    marca_name = None

    # Identifica a marca no nome do produto, com base na lista BRANDS
    for i, tok in enumerate(toks):
        tok_clean = re.sub(r"[\-\+]", " ", tok)
        for brand in BRANDS:
            if tok.lower() == brand.lower() or tok_clean.lower() == brand.lower():
                marca_idx = i
                marca_name = brand
                break
        if marca_idx is not None:
            break

    # Extrai categoria e modelo com base na posição da marca
    if marca_idx is not None:
        categoria = " ".join(toks[:marca_idx]).strip() or None
        modelo = " ".join(toks[marca_idx+1:]).strip() or None
        return {"categoria": categoria, "marca": marca_name, "modelo": modelo}
    else:
        categoria = toks[0]
        modelo = " ".join(toks[1:]).strip() or None
        return {"categoria": categoria, "marca": None, "modelo": modelo}

# Função para substituir valores vazios por nulos
def null_if_empty(col):
    return F.when(F.trim(col) == "", None).otherwise(col)

# Função para enriquecer o DataFrame de produtos
def enriquece_produtos(df, BRANDS):
    """
    Enriquecer DataFrame de produtos com categoria, marca, modelo, capacidade, dimensões, potência e memória.
    
    Parâmetros:
        df : PySpark DataFrame
            DataFrame de entrada com pelo menos as colunas 'nome_produto' e 'descricao'.
    
    Retorna:
        PySpark DataFrame com colunas adicionadas:
        ['categoria', 'marca', 'modelo', 'capacidade', 'dimensao', 'potencia']
    """
    # Extrai categoria, marca e modelo do nome do produto
    df_enriquecido = df.withColumn(
        "cat_brand_model",
        extract_cat_brand_model(F.col("nome_produto"))
    ).withColumn("categoria", F.col("cat_brand_model.categoria")) \
     .withColumn("marca", F.col("cat_brand_model.marca")) \
     .withColumn("modelo", F.col("cat_brand_model.modelo")) \
     .drop("cat_brand_model")

    # Cria um padrão regex para corresponder a qualquer nome de marca
    brands_pattern = '|'.join(BRANDS)

    # Atualiza o DataFrame para remover nomes de marcas da coluna 'nome_produto'
    df_enriquecido = df_enriquecido.withColumn(
        "nome_produto_cleaned",
        F.regexp_replace(F.col("nome_produto"), f"\\b({brands_pattern})\\b", "")
    )
    
    # Atualiza a coluna 'categoria' com base no nome do produto limpo
    # A Categoria consiste em strings de até 2 palavras (3 se conter stop-words) que descrevem em termos gerais, a categoria do produto
    df_enriquecido = df_enriquecido.withColumn(
        "categoria",
        F.when(
            (F.size(F.split(F.col("nome_produto_cleaned"), " ")) > 3) & 
            (F.array_contains(F.array(F.lit("de"), F.lit("da"), F.lit("do"), F.lit("das"), F.lit("dos"), F.lit("a"), F.lit("o"), F.lit("e")), 
                            F.split(F.col("nome_produto_cleaned"), " ")[1])),
            F.concat_ws(" ", F.slice(F.split(F.col("nome_produto_cleaned"), " "), 1, 3))
        ).when(
            F.size(F.split(F.col("nome_produto_cleaned"), " ")) > 2,
            F.concat_ws(" ", F.slice(F.split(F.col("nome_produto_cleaned"), " "), 1, 2))
        ).otherwise(F.col("nome_produto_cleaned"))
    )

    # Extrai capacidade em litros da descrição ou nome do produto
    df_enriquecido = df_enriquecido.withColumn(
        "capacidade",
        F.coalesce(
            null_if_empty(F.regexp_extract(F.col("descricao"), r"(\d+(?:,\d+)?\s?[lL])", 1)),
            null_if_empty(F.regexp_extract(F.col("nome_produto"), r"(\d+(?:,\d+)?\s?[lL])", 1))
        )
    ).withColumn(
        # Extrai dimensões da descrição ou nome do produto
        "dimensao",
        F.coalesce(
            null_if_empty(F.regexp_extract(F.col("descricao"), r"(\d+(?:,\d+)?\s?cm)", 1)),
            null_if_empty(F.regexp_extract(F.col("nome_produto"), r"(\d+(?:,\d+)?\s?cm)", 1)),
            null_if_empty(F.regexp_extract(F.col("descricao"), r"(\d+(?:\.\d+)?\s*(\"|”|''|pol))", 1)),
            null_if_empty(F.regexp_extract(F.col("nome_produto"), r"(\d+(?:\.\d+)?\s*(\"|”|''|pol))", 1))
        )
    ).withColumn(
        # Extrai potência em watts da descrição ou nome do produto
        "potencia",
        F.coalesce(
            null_if_empty(F.regexp_extract(F.col("descricao"), r"(\d+(?:,\d+)?\s?W)", 1)),
            null_if_empty(F.regexp_extract(F.col("nome_produto"), r"(\d+(?:,\d+)?\s?W)", 1))
        )
    )

    # Seleciona e re-ordena colunas
    df_enriquecido = df_enriquecido.select(
        'id_venda', 'canal_venda', 'valor', 'data_venda', 'ano_mes_venda',
        'id_produto', 'nome_produto', 'descricao', 'categoria', 'marca', 'modelo', 'capacidade', 'dimensao', 'potencia',
        'id_cliente', 'nome', 'email', 'telefone', 'estado', 'data_cadastro', 'categoria_produto'
    )

    return df_enriquecido

# Lista de marcas conhecidas
BRANDS = ["Mondial", "Dell", "Samsung", "LG", "Nespresso", "Arno", "Electrolux", "Brastemp", 
          "Black+Decker", "HP", "Lorenzetti", "Fischer", "Philips Walita", "Redragon", 
          "Logitech", "Xiaomi", "Apple", "Motorola", "Acer", "ThunderX3", "Fifine", 
          "Lenovo", "JBL", "Kingston", "Seagate", "TP-Link", "Clamper", "Britânia", "WAP"]

df_enriquecido = enriquece_produtos(df_unificado, BRANDS)

#### Tratamento da coluna Categoria_Produto (marca na tabela origem) e Categoria


In [0]:
df_enriquecido.select("categoria_produto","categoria").display()

##### Análise coluna Categoria_Produto e Categoria

- A coluna Categoria_Produto consiste na coluna "marca" corrigida e renomeada. A coluna "marca", na origem, continha informações sobre a categoria do produto, ou seja, se o produto fosse um Notebook Dell Inspiron 15, "marca" seria Notebook. Identificou-se que existinham valores nulos na origem para esse campo, esses valores foram preenchidos com a primeira palavra da coluna "nome_produto".
- A coluna Categoria é preenchida pela analise textual da coluna "nome_produto", a lógica por trás do preenchimento é: Identificar a marca do protudo, sua categoria será o que está antes da marca.
- Ao comparar as duas colunas, nota-se que a coluna contruida "Categoria", contém os mesmo dados que a coluna da origem "Categoria_Produto", sendo os dados da coluna "Categoria" mais completos e acertivos
- Assim, optou-se pela remoção da coluna "Categoria_Produto" da base final.

**Conclusão**: Remover cooluna "Categoria_Produto" ou "marca" do dataset final

In [0]:
df_enriquecido = df_enriquecido.drop("categoria_produto")

### Validação da Base Final

In [0]:
print("Tabela Enriquecida: ")
show_table(df_enriquecido)
basic_eda(df_enriquecido)
graphical_eda(df_enriquecido)

### Construção dos Indicadores

- Produtos mais vendidos;
- Faturamento por mês, canal, estado;
- Ticket médio;
- Sazonalidade;

In [0]:
def aggregate_product_data(df):
    # Seleciona as colunas relevantes
    df = df.select('id_venda', 'valor', 'id_produto', 'nome_produto')

    # Calcula o total de vendas e a soma total dos valores
    total_count = df.count()
    total_sum_valor = df.agg(F.sum('valor')).collect()[0][0]

    # Agrupa por nome_produto e calcula a quantidade de vendas e arrecadação
    df_agg = df.groupBy('id_produto', 'nome_produto').agg(
        F.count('id_venda').alias('qtd_vendas'),
        F.round(F.sum('valor'), 2).alias('arrecadacao'),
    ).withColumn(
        'percentual_vendas', F.round((F.col('qtd_vendas') / total_count) * 100, 2)
    ).withColumn(
        'percentual_arrecadacao', F.round((F.col('arrecadacao') / total_sum_valor) * 100, 2)
    )

    # Seleciona e ordena as colunas finais
    df_agg = df_agg.select(
        'id_produto', 'nome_produto', 'qtd_vendas', 'percentual_vendas', 'arrecadacao', 'percentual_arrecadacao'
    ).orderBy('qtd_vendas', ascending=False)

    return df_agg

df_prod_agg = aggregate_product_data(df_enriquecido)
df_prod_agg.display()

In [0]:
df_canal_estado = df_enriquecido.groupBy("canal_venda", "estado", "data_venda").agg(
    F.count("id_venda").alias("qtd_vendas"),
    F.round(F.sum("valor"), 2).alias("faturamento"),
    F.round(F.avg("valor"), 2).alias("ticket_medio")
).orderBy("faturamento")
df_canal_estado.display()

# Sazonalidade (vendas por mês)
df_sazonalidade = df_enriquecido.groupBy("data_venda").agg(
    F.count("id_venda").alias("qtd_vendas"),
    F.round(F.sum("valor"), 2).alias("faturamento"),
    F.round(F.avg("valor"), 2).alias("ticket_medio")
).orderBy("data_venda")
df_sazonalidade.display()